In [ ]:
from lahman_class import Lahman
from postgresql_class import PSQL

In [ ]:
lahman=Lahman()

In [ ]:
# gets directories/schemas from lahman github
schemas = lahman.get_directories()
# gets table names in each schema and returns in dict
layout = lahman.get_tbl_names_in_dirs(schemas)

In [ ]:
# prints schemas/directores and tables/csvs in them
for k, v in layout.items():
    print(k)
    print(v)
    print('\n')

In [ ]:
# create new DB named lahman
new_db_name = 'lahman'
print('password:')
password = input()
temp_db_engine = PSQL(user='postgres', password=password)
temp_db_engine.create_new_db(new_db_name)

In [ ]:
# connect to newly created db
print('password:')
password=input()
new_db_engine = PSQL(user='postgres', password=password, database=new_db_name) 

In [ ]:
# create schemas in database based on schema/directory list returned from github
new_db_engine.create_schemas(schemas)

In [ ]:
# loops through schemas and list of tables in them
# creates a table and bulk inserts data from df
temp_dict = {}
for schema, tables in layout.items():
    temp_dict = lahman.get_tbls_from_dir(tables=tables, directory=schema)
    for tbl_name, dataframe in temp_dict.items():
        new_db_engine.create_tbl(dataframe=dataframe, tbl_name=tbl_name, schema=schema)
        new_db_engine.insert_into_tbl(dataframe=dataframe, tbl_name=tbl_name, schema=schema)

In [ ]:
from postgresql_class import PSQL
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
print('user:')
user = input()
print('password:')
password = input()
print('database:')
database = input()

lahmandb = PSQL(user=user, password=password, database=database)

In [ ]:
query = """
SELECT
CONCAT(p."nameFirst" , ' ', p."nameLast") as "fullName",
b."yearID",
b."HR",
b."SB"
FROM core.batting b
LEFT JOIN core.people p
on b."playerID" = p."playerID" 
where p."nameLast" = 'Bonds'
and p."nameFirst" = 'Barry'
order by b."yearID" asc"""

In [ ]:
df = lahmandb.query_db(query)

In [ ]:
df

In [ ]:
df.set_index('yearID') # set x-axis

fig = plt.figure()

x = fig.add_subplot(111)
df['HR'].plot(ax=x, label='HR')
df['SB'].plot(ax=x, label='SB')

plt.show()